In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.mixture import GMM
from sklearn.decomposition import PCA
from load_data import load_data
from bacteriopop_utils import extract_features
import feature_selection_utils
import bacteriopop_utils

In [2]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GMM

In [35]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np

In [3]:
df = load_data()

In [4]:
df = extract_features(df)

In [5]:
df.head()

,abundance,class=Acidobacteriia,class=Actinobacteria,class=Alphaproteobacteria,class=Anaerolineae,class=Aquificae,class=Archaeoglobi,class=Bacilli,class=Bacteroidia,class=Betaproteobacteria,...,phylum=Proteobacteria,phylum=Spirochaetes,phylum=Synergistetes,phylum=Tenericutes,phylum=Thaumarchaeota,phylum=Thermodesulfobacteria,phylum=Thermotogae,phylum=Verrucomicrobia,replicate,week
0,0.228531,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,4
1,0.220860,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,1,4
2,0.054719,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,4
3,0.047956,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,4
4,0.040903,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,4


In [6]:
loaded_data = load_data()

In [7]:
extracted_features = bacteriopop_utils.extract_features(
    dataframe = loaded_data,
    column_list = ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'oxygen', 'abundance']
    # default list was: ['kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'length', 'abundance', 'project']
    )

In [8]:
extracted_features.head()

,abundance,class=Acidobacteriia,class=Actinobacteria,class=Alphaproteobacteria,class=Anaerolineae,class=Aquificae,class=Archaeoglobi,class=Bacilli,class=Bacteroidia,class=Betaproteobacteria,...,phylum=Planctomycetes,phylum=Poribacteria,phylum=Proteobacteria,phylum=Spirochaetes,phylum=Synergistetes,phylum=Tenericutes,phylum=Thaumarchaeota,phylum=Thermodesulfobacteria,phylum=Thermotogae,phylum=Verrucomicrobia
0,0.228531,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0.220860,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
2,0.054719,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.047956,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0.040903,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [57]:
df_pca = PCA(n_components=10).fit_transform(extracted_features)

In [58]:
df_pca.shape

(64755, 10)

In [54]:
target = df['abundance']

In [59]:
def make_ellipses(gmm, ax):
    for n, color in enumerate('rgb'):
        v, w = np.linalg.eigh(gmm._get_covars()[n][:2, :2])
        u = w[0] / np.linalg.norm(w[0])
        angle = np.arctan2(u[1], u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        v *= 9
        ell = mpl.patches.Ellipse(gmm.means_[n, :2], v[0], v[1],
                                  180 + angle, color=color)
        ell.set_clip_box(ax.bbox)
        ell.set_alpha(0.5)
        ax.add_artist(ell)

In [60]:
skf = StratifiedKFold(target, n_folds=4)
train_index, test_index = next(iter(skf))

In [61]:
X_train = df_pca[train_index]
y_train = target[train_index]
X_test = df_pca[test_index]
y_test = target[test_index]

In [62]:
n_classes = len(np.unique(y_train))

In [63]:
classifiers = dict((covar_type, GMM(n_components=n_classes,
                    covariance_type=covar_type, init_params='wc', n_iter=20))
                   for covar_type in ['spherical', 'diag', 'tied', 'full'])

In [64]:
n_classifiers = len(classifiers)

In [65]:
plt.figure(figsize=(3 * n_classifiers / 2, 6))
plt.subplots_adjust(bottom=.01, top=0.95, hspace=.15, wspace=.05,
                    left=.01, right=.99)

In [ ]:
0